# Test hfl_core.py

In [2]:
import numpy as np
import hfl_core

w_ofNid1 = [ np.array([[4, 0], [4, 0]], dtype=np.float32), np.array([8, 0, 8], dtype=np.float32) ]
w_ofNid2 = [ np.array([[0, 4], [0, 4]], dtype=np.float32), np.array([0, 8, 0], dtype=np.float32) ]
w_byNid = [ w_ofNid1, w_ofNid2 ]
weight_byNid = [ 1, 3 ]
w_avg = hfl_core.federated_aggregate1(w_byNid, weight_byNid)

assert( w_avg[0].tolist() == [[1, 3], [1, 3]] )
assert( w_avg[1].tolist() == [2, 6, 2] )
assert( type(w_avg[0][0,0]) == np.float32 )
assert( np.linalg.norm(hfl_core.np_flatten(w_ofNid1) - hfl_core.np_flatten(w_ofNid2)) == 16 )
print('Completed')

Completed


# Test hfl_alg.py

In [3]:
import tensorflow as tf
import numpy as np
import hfl_core
import hfl_alg
import hfl_struct
import hfl_util

assert( hfl_alg.calc_Q(2, 2, 0.5, 2, 0.5, 2, 0.5, 2) == 8 )

trainData, testData = tf.keras.datasets.mnist.load_data()
(trainData_byNid, train_z) = hfl_util.groupByEdge('cnn', 'mnist-o', trainData, 'one', 'one', 100, 10, True)

g_is__w = w_byNid
weight_byNid = [ 1, 3 ]
g__w = hfl_core.federated_aggregate1(g_is__w, weight_byNid)

numNodes = 2
numGroups = 2
z = [0, 1]
testData_byNid = np.array([ trainData_byNid[i] for i in range(numNodes) ])
c = hfl_struct.Cloud(None, np.array([ trainData_byNid[i] for i in range(numNodes) ]), numGroups)
c.digest(z, debugging=True)

# 1. deltaMap 을 이용한 delta 구하기
(deltaMap, DeltaMap) = hfl_alg.calc_deltaMap(c, g_is__w, g__w, numNodes)
assert( deltaMap == [[0.0, 16.0], [16.0, 0.0]] )
assert( DeltaMap == [[12.0, 4.0], [12.0, 4.0]] )

(delta1, Delta1) = hfl_alg.calc_delta(c, deltaMap, DeltaMap)

# 2. deltaMap 없이 delta 구하기
def federated_delta(g_global, g_local_byId, weight_byId):
    return np.average([ hfl_core.np_normOfDiff(g_global, g_local) for g_local in g_local_byId ], weights=weight_byId)
g_ks__w = [] ; idxBegin = 0 ; idxEnd = 0
for g in c.groups:
    idxEnd += len(g.get_N_k())
    g_k_is__w = g_is__w[idxBegin:idxEnd]
    g_k__w = hfl_core.federated_aggregate1(g_k_is__w, g.get_D_k_is())
    g_ks__w.append(g_k__w)
    idxBegin = idxEnd
    
delta_ks2 = [] ; idxBegin = 0 ; idxEnd = 0
for k, g in enumerate(c.groups):
    idxEnd += len(g.get_N_k())
    g_k_is__w = g_is__w[idxBegin:idxEnd]
    g_k__w = g_ks__w[k]
    delta_k2 = federated_delta(g_k__w, g_k_is__w, g.get_D_k_is())
    delta_ks2.append(delta_k2)
    idxBegin = idxEnd
delta2 = np.average(delta_ks2, weights=c.get_D_ks())
Delta2 = federated_delta(g__w, g_ks__w, c.get_D_ks())
assert( delta1 == delta2 )
assert( Delta1 == Delta2 )
print('Completed')

AttributeError: module 'hfl_alg' has no attribute 'calc_Q'